In [ ]:
# Install required packages
!pip install -q gradio python-dotenv google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 107.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.8 MB/s eta 0:00:00


In [ ]:
!pip install langdetect

In [ ]:
import gradio as gr
import os
import sqlite3
import google.generativeai as genai
import json
from datetime import datetime
import tempfile
import requests
from google.colab import files

from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

# Function to create or download the database
def setup_database():
    """Downloads or creates the Northwind database file and returns the path"""
    # Define paths
    db_dir = "./database"
    db_path = f"{db_dir}/Northwind.db"

    # Create directory if it doesn't exist
    if not os.path.exists(db_dir):
        os.makedirs(db_dir)

    # Check if DB already exists
    if os.path.exists(db_path):
        print(f"Database already exists at {db_path}")
        return db_path

    #Allow user to upload a database file
    print("Please upload your Northwind.db file")
    uploaded = files.upload()
    if uploaded:
        # Get the filename of the uploaded file
        filename = list(uploaded.keys())[0]
        # Move the file to the desired location
        with open(db_path, 'wb') as f:
            f.write(uploaded[filename])
        print(f"Database uploaded to {db_path}")
        return db_path

# Define the response schema for structured output
response_schema = {
    "type": "object",
    "properties": {
        "query_understanding": {
            "type": "string",
            "description": "Brief explanation of how the system interpreted the user's question"
        },
        "sql_query": {
            "type": "string",
            "description": "The generated SQL query to execute on the database"
        },
        "execution_status": {
            "type": "string",
            "enum": ["success", "error"],
            "description": "Status of the query execution"
        },
        "error_message": {
            "type": "string",
            "description": "Error message if the execution failed"
        },
    },
    "required": ["query_understanding", "sql_query", "execution_status"]
}

# Function to get Gemini response using chat model
def get_gemini_chat_response(question, system_prompt):
    # Initialize the chat model
    model = genai.GenerativeModel(
        'gemini-1.5-pro',
        generation_config={
            "temperature": 0.1,
            "top_p": 0.95,
            "response_mime_type": "application/json",
        }
    )

    # Start a chat session
    chat = model.start_chat(
        history=[{"role": "user", "parts": [system_prompt]}]
    )

    try:
        # Send the user query to the chat session
        response = chat.send_message(
            question,
            generation_config={"response_schema": response_schema}
        )

        # Print JSON response to terminal
        print("\n=== JSON RESPONSE FROM GEMINI ===")
        print(response.text)
        print("===============================\n")

        return response.text
    except Exception as e:
        error_response = json.dumps({
            "query_understanding": "Error processing query",
            "sql_query": "",
            "execution_status": "error",
            "error_message": str(e),
            "explanation": "There was an error processing your query. Please try again."
        })
        print("\n=== ERROR RESPONSE ===")
        print(error_response)
        print("=====================\n")
        return error_response

# Function to execute SQL query
def execute_sql_query(sql_query, db_path):
    try:
        conn = sqlite3.connect(db_path)
        cur = conn.cursor()
        cur.execute(sql_query)
        rows = cur.fetchall()
        column_names = [desc[0] for desc in cur.description]  # Get column names
        conn.close()

        # Convert query results into JSON structure
        results = [dict(zip(column_names, row)) for row in rows]

        # Print SQL results to terminal in JSON format
        print("\n=== SQL EXECUTION RESULTS ===")
        print(json.dumps({
            "status": "success",
            "sql_query": sql_query,
            "row_count": len(rows),
            "sample": results[0] if results else "No results"
        }, indent=2))
        print("===========================\n")

        return {
            "status": "success",
            "sql_query": sql_query,
            "columns": column_names,
            "rows": results
        }
    except Exception as e:
        print("\n=== SQL ERROR ===")
        print(json.dumps({
            "status": "error",
            "sql_query": sql_query,
            "error_message": str(e)
        }, indent=2))
        print("================\n")

        return {
            "status": "error",
            "sql_query": sql_query,
            "error_message": str(e)
        }

# Main function to process user query
def process_query(question, db_path, history=None):


    # Print user query to terminal
    print(f"\n>>> USER QUERY: {question}")

    # For empty initial messages, return a welcome message
    if not question.strip():
        return json.dumps({
            "response_type": "welcome",
            "message": "Welcome, manager! How can I help you today?",
        }, indent=2)

    # Enhanced system prompt with date functions and complex query examples
    system_prompt = """
You are NorthwindAssistant, a database assistant that helps managers query the Northwind database. Your job is to:
1. Understand questions in English or Turkish and answer them in the same language
2. Convert them to SQL queries that work specifically with SQLite syntax
3. Return results in structured JSON format

IMPORTANT:
- For date calculations, use SQLite date functions like julianday(), date(), strftime()
- For date comparisons, ensure proper format conversion
- Use proper SQL aggregate functions (AVG, MAX, MIN, COUNT, SUM)
- Always check table relationships before joining

DATABASE SCHEMA:

Categories: CategoryID, CategoryName, Description
Customers: CustomerID, CustomerName, ContactName, Address, City, PostalCode, Country
Employees: EmployeeID, LastName, FirstName, BirthDate (TEXT format: 'YYYY-MM-DD'), Photo, Notes
Shippers: ShipperID, ShipperName, Phone
Suppliers: SupplierID, SupplierName, ContactName, Address, City, PostalCode, Country, Phone
Products: ProductID, ProductName, SupplierID, CategoryID, Unit, Price
Orders: OrderID, CustomerID, EmployeeID, OrderDate (DATETIME format), ShipperID
Order Details: OrderDetailID, OrderID, ProductID, Quantity

EXAMPLE QUERIES:

1. "What's the average price of all products?"
SQL: SELECT AVG(Price) as AveragePrice FROM Products;

2. "Which employee's birthday is closest to today?"
SQL: SELECT EmployeeID, FirstName, LastName, BirthDate FROM Employees ORDER BY ABS(julianday(strftime('%m-%d', BirthDate)) - julianday(strftime('%m-%d', 'now'))) ASC LIMIT 1;

3. "Show me the total sales by country in 2023"
SQL: SELECT c.Country, SUM(od.Quantity * p.Price) as TotalSales FROM Orders o
JOIN Customers c ON o.CustomerID = c.CustomerID
JOIN "Order Details" od ON o.OrderID = od.OrderID
JOIN Products p ON od.ProductID = p.ProductID
WHERE strftime('%Y', o.OrderDate) = '2023' GROUP BY c.Country ORDER BY TotalSales DESC;

4. "Who are our top 5 employees by sales?"
SQL: SELECT e.EmployeeID, e.FirstName, e.LastName, SUM(od.Quantity * p.Price) as TotalSales
FROM Employees e
JOIN Orders o ON e.EmployeeID = o.EmployeeID
JOIN "Order Details" od ON o.OrderID = od.OrderID
JOIN Products p ON od.ProductID = p.ProductID
GROUP BY e.EmployeeID ORDER BY TotalSales DESC LIMIT 5;

5. "En çok satış yapılan 3 ülke hangisidir?" (What are the top 3 countries by sales?)
SQL: SELECT c.Country, COUNT(o.OrderID) as OrderCount FROM Orders o JOIN Customers c ON o.CustomerID = c.CustomerID GROUP BY c.Country ORDER BY OrderCount DESC LIMIT 3;

For any user question, respond with this JSON structure:
{
  "query_understanding": "Brief explanation of the user's question",
  "sql_query": "SQL query to execute",
  "execution_status": "success",
  "error_message": "",
  "explanation": "Explanation of the results in user's language"
}

If you can't generate a valid SQL query, set execution_status to "error" and provide an error_message.

Remember to handle SQLite's specific date formats and functions correctly when dealing with date calculations.
Don't edit/delete any tables/columns/rows from the database.
"""

    # Get the response from Gemini chat
    try:
        llm_response = get_gemini_chat_response(question, system_prompt)
        llm_data = json.loads(llm_response)

        # Extract the SQL query
        sql_query = llm_data.get("sql_query", "")

        # If no SQL query or status is error, return the explanation from the LLM
        if not sql_query or llm_data.get("execution_status") == "error":
            error_response = json.dumps({
                "response_type": "error",
                "message": llm_data.get("explanation", "I couldn't generate a SQL query for your question. Could you please rephrase it?")
            }, indent=2)
            print("\n=== ERROR RESPONSE ===")
            print(error_response)
            print("=====================\n")
            return error_response

        # Execute the SQL query
        result = execute_sql_query(sql_query, db_path)

        # Create the combined response
        final_response = json.dumps({
            "response_type": "query_result",
            "understanding": llm_data.get("query_understanding", ""),
            "query": sql_query,
            "result": result
        }, indent=2)

        # Print final response to terminal
        print("\n=== FINAL RESPONSE ===")
        print(final_response)
        print("=====================\n")

        return final_response

    except Exception as e:
        error_msg = f"Error processing the request: {str(e)}"
        error_response = json.dumps({
            "response_type": "error",
            "message": error_msg,
            "raw_response": llm_response if 'llm_response' in locals() else "No response generated"
        }, indent=2)
        print("\n=== PROCESS ERROR ===")
        print(error_response)
        print("====================\n")
        return error_response

# Define the respond function outside of the Blocks context
def respond(message, chat_history, db_path):
    # Process the query
    response = process_query(message, db_path)

    try:
        # Parse the JSON response from the LLM
        response_data = json.loads(response)
        response_type = response_data.get("response_type", "")


        # Add user message to history if not empty
        if message.strip():
            chat_history.append({"role": "user", "content": message})

        if response_type == "welcome" or response_type == "clarification":
            # Simple welcome or clarification response
            bot_message = response_data.get("message", "")

        elif response_type == "query_result":
            # We have a successful query result
            result = response_data.get("result", {})

            if result.get("status") == "success":
                rows = result.get("rows", [])
                columns = result.get("columns", [])

                # Start building a user-friendly answer
                bot_message = ""

                # 1) Provide a quick sentence about the result
                #    If there's only 1 row & 1 column, we can say "The X is Y."
                if len(columns) == 1 and len(rows) == 1:
                    col_name = columns[0]
                    value = rows[0].get(col_name, "")
                    bot_message += f"The **{col_name}** is **{value}**.\n\n"
                else:
                    # Otherwise, just say we have multiple results
                    bot_message += "Here are your query results:\n\n"

                # 2) Show the data in a table
                if columns:
                    # Table headers
                    bot_message += "| " + " | ".join(columns) + " |\n"
                    bot_message += "| " + " | ".join(["---" for _ in columns]) + " |\n"

                    # Table rows (no limit)
                for row in rows:
                    bot_message += "| " + " | ".join(str(row.get(col, "")) for col in columns) + " |\n"


                # 3) Finally, add the explanation
                explanation = response_data.get("explanation", "")
                if explanation:
                    bot_message += f"\n**Explanation**: {explanation}"

            else:
                # If the SQL execution failed
                bot_message = f"Error executing query: {result.get('error_message', 'Unknown error')}"

        elif response_type == "error":
            # LLM couldn't generate a query or something else went wrong
            bot_message = response_data.get("message", "I couldn't process your request properly.")

        else:
            # Fallback if we don't recognize the response type
            bot_message = "I couldn't process your request properly."

        # Add the final bot message to the chat history
        chat_history.append({"role": "assistant", "content": bot_message})

        # Return the updated history and clear the user input box
        return chat_history, ""

    except Exception as e:
        # If something goes wrong in formatting the response
        error_msg = f"Error processing response: {str(e)}"
        print("\n=== RESPONSE FORMATTING ERROR ===")
        print(json.dumps({"error": error_msg}, indent=2))
        print("===============================\n")

        # Add to chat history for debugging
        chat_history.append({"role": "assistant", "content": error_msg})
        return chat_history, ""


# Function to initialize the chat with a welcome message
def init_chat():
    chat_history = []
    chat_history.append({"role": "assistant", "content": "Welcome, manager! How can I help you today? "})
    return chat_history, ""

# Create the Gradio interface with database path as a state variable
def create_demo(db_path):
    with gr.Blocks() as demo:
        gr.Markdown("# Database Chatbot Assistant")

        chatbot = gr.Chatbot(type="messages")
        msg = gr.Textbox(placeholder="Ask about the Northwind database...")
        clear = gr.Button("Clear")

        # Set up event handlers within the Blocks context
        # Use a partial application to include the database path
        msg.submit(lambda message, history: respond(message, history, db_path), [msg, chatbot], [chatbot, msg])
        clear.click(lambda: ([], ""), None, [chatbot, msg])

        # Initialize the chatbot with a welcome message
        demo.load(init_chat, None, [chatbot, msg])

    return demo

# Main function to set up and run everything
def main():
    print("Setting up Database Chatbot Assistant for Google Colab...")

    # Set up database
    db_path = setup_database()

    # Create and launch the demo
    print("Starting Database Chatbot Assistant...")
    print("Terminal will display JSON responses for monitoring")
    demo = create_demo(db_path)
    demo.launch(debug=True, share=True)  # share=True creates a public link

# Run the application
if __name__ == "__main__":
    main()

Setting up Database Chatbot Assistant for Google Colab...
Database already exists at ./database/Northwind.db
Starting Database Chatbot Assistant...
Terminal will display JSON responses for monitoring
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://05a3f0dd3112090668.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



>>> USER QUERY: delete employees table

=== JSON RESPONSE FROM GEMINI ===
{"execution_status": "error", "query_understanding": "The user requested to delete the employees table.", "sql_query": "", "error_message": "I am not allowed to delete or modify the database schema."}


=== ERROR RESPONSE ===
{
  "response_type": "error",
  "message": "I couldn't generate a SQL query for your question. Could you please rephrase it?"
}


>>> USER QUERY: list all categories

=== JSON RESPONSE FROM GEMINI ===
{"execution_status": "success", "query_understanding": "Retrieve all categories from the Categories table.", "sql_query": "SELECT * FROM Categories;", "error_message": ""}


=== SQL EXECUTION RESULTS ===
{
  "status": "success",
  "sql_query": "SELECT * FROM Categories;",
  "row_count": 8,
  "sample": {
    "CategoryID": 1,
    "CategoryName": "Beverages",
    "Description": "Soft drinks, coffees, teas, beers, and ales"
  }
}


=== FINAL RESPONSE ===
{
  "response_type": "query_result",
  "und